In [ ]:
# Install
#!pip install matplotlib
#!pip install pandas
#!pip install mne
#%pip install PyQt6 
#%pip install pyqtgraph
import sys
print(sys.version)

3.11.1 (tags/v3.11.1:a7a450f, Dec  6 2022, 19:58:39) [MSC v.1934 64 bit (AMD64)]


In [ ]:
import sys
sys.path.append('C:/Python37/Lib/site-packages')

from IPython.display import clear_output
#%matplotlib qt5
#%matplotlib inline


from pyqtgraph.Qt import QtGui, QtCore
import pyqtgraph as pg
import random
from pyOpenBCI import OpenBCICyton
import threading
import time
import numpy as np
from scipy import signal

In [ ]:
from pyOpenBCI import OpenBCICyton

def print_raw(sample):
    print(sample.channels_data)

board = OpenBCICyton(port='COM8', daisy=False)

def start_cyton():
    try:
        board.start_stream(print_raw)
    except:
        pass

y = threading.Thread(target=start_cyton)
y.daemon = True
y.start()

time.sleep(.02)
board.disconnect()

Serial established
Skipped 4 bytes before start found
[680963, -821127, -863262, -852887, 816924, 403656, 735099, 425999]
[-29872, -852099, -700393, -713156, 710588, 1134108, 486581, 1222660]
[-955904, -481447, -341007, -354342, 1092810, 2729431, 773174, 2535600]
[-312106, -316024, -530052, -515351, 1251351, 2178225, 1061044, 1880524]
[664000, -636597, -914144, -894097, 884031, 509532, 802991, 485593]
[164723, -885068, -757224, -765051, 694474, 853160, 496149, 963104]
[-890597, -627062, -338771, -362140, 1018336, 2551538, 696337, 2409887]
[-540035, -392766, -401415, -389497, 1274319, 2471767, 1053558, 2153028]
[588229, -739367, -788111, -760693, 960856, 730914, 882318, 648664]
[340933, -858923, -814076, -820331, 694831, 647313, 534209, 758763]
[-762083, -661981, -407418, -430580, 939213, 2305454, 631299, 2215091]
[-716641, -372332, -348243, -345208, 1272612, 2684918, 1024310, 2359589]
[459848, -609359, -770158, -742212, 1032064, 987543, 946949, 847413]
[474554, -867134, -848592, -84842

Above code from github. Use to test whether live data is streaming.

In [ ]:
from pyqtgraph.Qt import QtWidgets
import pyqtgraph as pg

# Constants
SCALE_FACTOR = (4500000) / 24 / (2**23 - 1)  # From the pyOpenBCI repo
colors = 'rgbycmwr'

# Create PyQt application and window
app = QtWidgets.QApplication([])
win = pg.GraphicsLayoutWidget(show=True, title="N400 Real-Time ERP")

# Create plots for 8 EEG channels
ts_plots = []
for i in range(1, 9):
    p = win.addPlot(title=f'Channel {i}')
    p.setLabel('left', 'uV')
    ts_plots.append(p)
    win.nextRow()  # Move to next row after each plot

# Start the Qt event loop (if not embedded in something else)
app.exec_()


In [ ]:
from pyOpenBCI import OpenBCICyton
import numpy as np
import threading
import pyqtgraph as pg
from pyqtgraph.Qt import QtWidgets, QtCore
import sys
from collections import deque

# Constants
SCALE_FACTOR = (4500000) / 24 / (2**23 - 1)  # Converts raw int to microvolts
CHANNEL_COUNT = 8
BUFFER_SIZE = 250  # About 1–2 seconds of data depending on sampling rate

# Create a buffer for each channel
eeg_buffers = [deque([0]*BUFFER_SIZE, maxlen=BUFFER_SIZE) for _ in range(CHANNEL_COUNT)]

# Set up the Qt Application and plot window
app = QtWidgets.QApplication(sys.argv)
win = pg.GraphicsLayoutWidget(show=True, title="EEG Real-Time Plot")
win.resize(1000, 600)

# Create plots and curves
plots = []
curves = []

for i in range(CHANNEL_COUNT):
    p = win.addPlot(title=f"Channel {i+1}")
    p.setYRange(-100, 100)
    c = p.plot(pen=pg.intColor(i))
    plots.append(p)
    curves.append(c)

# Function to receive samples from Cyton board
def handle_sample(sample):
    for i in range(CHANNEL_COUNT):
        value = sample.channels_data[i] * SCALE_FACTOR
        eeg_buffers[i].append(value)

# Start the Cyton stream in a thread
def start_stream():
    board = OpenBCICyton(port='COM8', daisy=False)
    print("Initializing board...")
    board.start_stream(handle_sample)

stream_thread = threading.Thread(target=start_stream)
stream_thread.daemon = True
stream_thread.start()

# Timer to update plots every 50 ms
def update_plots():
    for i in range(CHANNEL_COUNT):
        curves[i].setData(list(eeg_buffers[i]))

timer = QtCore.QTimer()
timer.timeout.connect(update_plots)
timer.start(50)

# Start the Qt event loop
sys.exit(app.exec_())


Serial established
Initializing board...
Skipped 100 bytes before start found


KeyboardInterrupt: 

In [ ]:
# Define OpenBCI callback function
def save_data(sample):
    global data
    data.append([i*SCALE_FACTOR for i in sample.channels_data])

# Define function to update the graphs
def updater():
    global data, plots, colors
    t_data = np.array(data[-1000:]).T #transpose data
    fs = 250 #Hz

    # Notch Filter
    def notch_filter(val, data, fs=250):
        notch_freq_Hz = np.array([float(val)])
        for freq_Hz in np.nditer(notch_freq_Hz):
            bp_stop_Hz = freq_Hz + 3.0 * np.array([-1, 1])
            b, a = signal.butter(3, bp_stop_Hz / (fs / 2.0), 'bandstop')
            fin = data = signal.lfilter(b, a, data)
        return fin

    # Bandpass filter
    def bandpass(start, stop, data, fs = 250):
        bp_Hz = np.array([start, stop])
        b, a = signal.butter(5, bp_Hz / (fs / 2.0), btype='bandpass')
        return signal.lfilter(b, a, data, axis=0)

    # Applying the filters
    nf_data = [[],[],[],[],[],[],[],[]]
    bp_nf_data = [[],[],[],[],[],[],[],[]]

    for i in range(8):
        nf_data[i] = notch_filter(60, t_data[i])
        bp_nf_data[i] = bandpass(0.1, 30, nf_data[i])

    # Plot a time series of the raw data
    for j in range(8):
        ts_plots[j].clear()
        ts_plots[j].plot(pen=colors[j]).setData(t_data[j])

        # Define thread function
def start_board():
    board = OpenBCICyton(port='COM5', daisy=False)
    board.start_stream(save_data)
    
# Initialize Board and graphing update
if __name__ == '__main__':
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        x = threading.Thread(target=start_board)
        x.daemon = True
        x.start()

        timer = QtCore.QTimer()
        timer.timeout.connect(updater)
        timer.start(0)


        QtGui.QApplication.instance().exec_()


NameError: name 'sys' is not defined